In [1]:
import pyspark

In [2]:
pyspark.__file__

'/home/kenho/spark/spark-3.3.2-bin-hadoop3/python/pyspark/__init__.py'

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[4]") \
    .appName('pyspark') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/09/03 11:41:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

In [4]:
df = spark.read.parquet('fhvhv_tripdata_2021-01.parquet')

In [5]:
df = df.repartition(24)

In [ ]:
df.write.parquet('fhvhv/2021/01/')

In [6]:
df = spark.read.parquet('fhvhv/2021/01/')

In [7]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

In [8]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID')\
    .filter(df.hvfhs_license_num == 'HV0003') \
    .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-11 18:40:22|2021-01-11 19:15:49|         262|         231|
|2021-01-05 15:13:22|2021-01-05 15:27:50|          61|         181|
|2021-01-31 18:42:09|2021-01-31 18:59:52|         232|           4|
|2021-01-27 22:24:36|2021-01-27 22:26:43|          68|          68|
|2021-01-30 08:35:46|2021-01-30 08:39:42|         256|         255|
|2021-01-16 02:25:35|2021-01-16 02:34:21|          89|          91|
|2021-01-11 11:58:23|2021-01-11 12:14:19|          97|          61|
|2021-01-03 07:44:58|2021-01-03 08:04:45|          26|         178|
|2021-01-14 18:52:00|2021-01-14 19:19:00|         181|         198|
|2021-01-08 20:35:35|2021-01-08 21:06:33|          76|          91|
|2021-01-15 13:49:48|2021-01-15 14:35:23|         246|          16|
|2021-01-27 10:37:56|2021-01-27 10:53:35|       

In [15]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

In [16]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-11|  2021-01-11|         262|         231|
| 2021-01-05|  2021-01-05|          61|         181|
| 2021-01-02|  2021-01-02|         100|           1|
| 2021-01-31|  2021-01-31|         232|           4|
| 2021-01-05|  2021-01-05|         162|           1|
| 2021-01-27|  2021-01-27|          68|          68|
| 2021-01-18|  2021-01-18|         205|         205|
| 2021-01-30|  2021-01-30|         256|         255|
| 2021-01-16|  2021-01-16|          89|          91|
| 2021-01-05|  2021-01-05|         132|         102|
| 2021-01-11|  2021-01-11|          97|          61|
| 2021-01-22|  2021-01-22|          79|          37|
| 2021-01-03|  2021-01-03|          26|         178|
| 2021-01-14|  2021-01-14|         181|         198|
| 2021-01-08|  2021-01-08|          76|          91|
| 2021-01-15|  2021-01-15|         246|       

In [17]:
def test_udf_sample(num):
    num_i = int(num[1:])
    if num_i % 7 == 0:
        return f's/{num_i:03x}'
    elif num_i % 3 == 0:
        return f'a/{num_i:03x}'
    else:
        return f'e/{num_i:03x}'

In [18]:
test_udf_sample('B02884')

's/b44'

In [19]:
test_udf = F.udf(test_udf_sample, StringType())

In [20]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', test_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

Traceback (most recent call last):
  File "/home/kenho/spark/spark-3.3.2-bin-hadoop3/python/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/kenho/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/home/kenho/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/kenho/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 692, in reducer_override
    return self._function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/kenho/spark/spark-3.3.2-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 565, in _function_reduce
    return self._dynamic_function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/k

PicklingError: Could not serialize object: IndexError: tuple index out of range